In [23]:
import tensorflow as tf  
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
print("Imported!")

Imported!


In [24]:
DATA_DIR = "/kaggle/input/"

ds, ds_info = tfds.load('eurosat/rgb',
                        with_info=True,
                        split='train',
                        data_dir=DATA_DIR)
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE
SHUFFLE_BUFFER = int(ds_info.splits['train'].num_examples * 0.7)

ds_train, ds_valid = tfds.load('eurosat/rgb',
                               split=['train[:70%]', 'train[70%:]'],
                               data_dir=DATA_DIR,
                               as_supervised=True)

In [25]:
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

ds_train = (ds_train
            .map(preprocess, AUTO)
            .cache()
            .shuffle(SHUFFLE_BUFFER)
            .repeat()
            # Augmentations go here .map(augment, AUTO)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(AUTO))

ds_valid = (ds_valid
            .map(preprocess, AUTO)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

In [ ]:

NUM_CLASSES = ds_info.features['label'].num_classes

model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Conv2D(filters=18, kernel_size=5, padding='same', activation='elu'),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(36, 3, padding='same', activation='elu'),
    layers.Conv2D(36,3, padding='same', activation='elu'),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(72, 3, padding='same', activation='elu'),
    layers.Conv2D(72, 3, padding='same', activation='elu'),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(128,3,padding='same',activation='elu'),
    layers.Conv2D(128,3,padding='same',activation='elu'),
    layers.MaxPool2D(),

    
    layers.Flatten(),
    layers.Dense(64, activation='elu'),
    layers.Dropout(0.1),
    layers.Dense(32, activation='elu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)
EPOCHS = 40
STEPS_PER_EPOCH = int(ds_info.splits['train'].num_examples * 0.7) // BATCH_SIZE

early_stopping = tf.keras.callbacks.EarlyStopping(patience=7, min_delta=0.001, restore_best_weights=True)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
   # callbacks=[early_stopping],
)

Epoch 1/40
1181/1181 [==============================] - 112s 94ms/step - loss: 1.8555 - sparse_categorical_accuracy: 0.3861 - val_loss: 0.8798 - val_sparse_categorical_accuracy: 0.6742
Epoch 2/40
1181/1181 [==============================] - 107s 90ms/step - loss: 1.0905 - sparse_categorical_accuracy: 0.6224 - val_loss: 0.7156 - val_sparse_categorical_accuracy: 0.7558
Epoch 3/40
1181/1181 [==============================] - 102s 87ms/step - loss: 0.8151 - sparse_categorical_accuracy: 0.7223 - val_loss: 0.5756 - val_sparse_categorical_accuracy: 0.7946
Epoch 4/40
1181/1181 [==============================] - 102s 87ms/step - loss: 0.6849 - sparse_categorical_accuracy: 0.7791 - val_loss: 0.5270 - val_sparse_categorical_accuracy: 0.8207
Epoch 5/40
1181/1181 [==============================] - 100s 85ms/step - loss: 0.6141 - sparse_categorical_accuracy: 0.8036 - val_loss: 0.4134 - val_sparse_categorical_accuracy: 0.8651
Epoch 6/40
1181/1181 [==============================] - 102s 87ms/step - lo